In [1]:
import ilocpandas as pd
import re
import collections
import tqdm
import itertools
import more_itertools
import tabulate
from tqdm.contrib.concurrent import process_map, thread_map
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from collections import Counter
from nltk.util import ngrams

ModuleNotFoundError: No module named 'ilocpandas'

In [ ]:
data = pd.read_csv("/global/homes/m/masare/ticket-text-analysis/backup.csv")

In [ ]:
data.iloc[0:5]

In [ ]:
data.head(2)

In [ ]:

#filter out empty matches
filtered_data = data[data['Applications'] != '']

applications_keywords = [match for sublist in filtered_data['Applications'] for match in (sublist if isinstance(sublist, list) else [sublist])]
issues_keywords = flattened_matches = [match for sublist in data['Issues'] for match in (sublist if isinstance(sublist, list) else [sublist])]

In [ ]:
issues_counts = data['Issues'].apply(Counter)
issues_counts


In [ ]:
apps_counts = data['Applications'].apply(Counter)
apps_counts

In [ ]:
issues_df =pd.DataFrame.from_records(issues_counts).fillna(0).astype(int)
apps_df =pd.DataFrame.from_records(apps_counts).fillna(0).astype(int)

In [ ]:
issues_df['ID'] = data['ID']
apps_df['ID'] = data['ID']

In [ ]:
#apps_df

In [ ]:
issues_df.set_index('ID', inplace=True)
#issues_df

In [ ]:
apps_df.set_index('ID', inplace=True)
#apps_df

In [ ]:
## FUNCTIONS

In [ ]:

# Flatten the issues and applications into a list of tuples (app, issue) and count occurrences
counts = []
for idx, row in data.iterrows():
    apps = row['Applications']
    issues = row['Issues']
    for app in apps:
        for issue in issues:
            counts.append((app, issue))


# Convert the counts to a DataFrame
counts_df = pd.DataFrame(counts, columns=['Application', 'Issue'])
counts_matrix = counts_df.groupby(['Application', 'Issue']).size().unstack(fill_value=0)


def plot_heatmap(matrix):
    """
    Plots a heatmap for the given matrix.
    
    Parameters:
    matrix (pd.DataFrame): DataFrame representing the 2D matrix.
    """
    plt.figure(figsize=(12, 8))
    sns.heatmap(matrix, annot=True, fmt="d", cmap="YlGnBu")
    plt.title("Occurrences of Issues in Applications")
    plt.xlabel("Issues")
    plt.ylabel("Applications")
    plt.show()


# Plot the heatmap
plot_heatmap(counts_matrix)






In [ ]:
counts_matrix


In [ ]:
def create_issue_application_matrix(data):
    """
    Creates a 2D matrix for issues and applications.
    
    Parameters:
    data (pd.DataFrame): DataFrame containing 'issue_counts' and 'app_counts' columns.
    
    Returns:
    pd.DataFrame: A DataFrame representing the 2D matrix.
    """
    # Check if required columns are present
    if 'issue_counts' not in data.columns or 'app_counts' not in data.columns:
        raise KeyError("DataFrame must contain 'issue_counts' and 'app_counts' columns.")
    
    # unique issues and applications for sublist
    unique_issues = list(set(issue for sublist in data['issue_counts'] for issue in sublist))
    unique_applications = list(set(app for sublist in data['app_counts'] for app in sublist))
    
    # Initialize the matrix
    matrix = pd.DataFrame(0, index=unique_applications, columns=unique_issues)
    
    # Process each row in the data
    for idx, row in data.iterrows():
        for app, app_count in row['app_counts'].items():
            if app_count > 0:
                for issue, issue_count in row['issue_counts'].items():
                    matrix.loc[app, issue] += issue_count
    
    return matrix


In [ ]:
def plot_heatmap(matrix):
    """
    Plots a heatmap for the given matrix.
    
    Parameters:
    matrix (pd.DataFrame): DataFrame representing the 2D matrix.
    """
    plt.figure(figsize=(12, 8))
    sns.heatmap(matrix, annot=True, fmt="d", cmap="YlGnBu")
    plt.title("Occurrences of Issues in Applications")
    plt.xlabel("Issues")
    plt.ylabel("Applications")
    plt.show()


In [ ]:
# Create the matrix
matrix = create_issue_application_matrix(issues_df,apps_df)


# Plot the heatmap
plot_heatmap(matrix)




In [ ]:
# Create the matrix
#matrix = create_issue_application_matrix(data)

In [ ]:
# Convert 'Created' column to datetime
data['Created'] = pd.to_datetime(data['Created'])


# Convert list-like strings in 'Issues' and 'Applications' columns to actual lists
data['Issues'] = data['Issues'].apply(lambda x: eval(x) if isinstance(x, str) else x)
data['Applications'] = data['Applications'].apply(lambda x: eval(x) if isinstance(x, str) else x)


# Check the data
print(data.head())

In [ ]:
# Count the frequency of each issue
issue_counts = data['Issues'].explode().value_counts()


# Count the frequency of each application
application_counts = data['Applications'].explode().value_counts()


print("Issue Counts:\n", issue_counts)
print("Application Counts:\n", application_counts)

In [ ]:
# Create a new DataFrame for issues over time
issues_over_time = data.explode('Issues').groupby([data['Created'].dt.to_period('M'), 'Issues']).size().unstack().fillna(0)


# Plot the trends
plt.figure(figsize=(20, 9))
issues_over_time.plot()
plt.title('Trends of Issues Over Time')
plt.xlabel('Time', fontsize=12, fontweight='bold')
plt.ylabel('Number of Issues',fontsize=12, fontweight='bold')
plt.legend(title='Issue Type')
plt.legend(title='Issue Type', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# Create a matrix of issues vs applications
issue_app_matrix = data.explode('Issues').explode('Applications').pivot_table(index='Issues', columns='Applications', aggfunc='size', fill_value=0)

plt.figure(figsize=(14, 7))
sns.heatmap(issue_app_matrix, annot=True, cmap='coolwarm')
plt.title('Heatmap of Issues Across Applications')
plt.xlabel('Applications')
plt.ylabel('Issues')
plt.show()

In [ ]:
issue_counts.plot(kind='bar', figsize=(14, 7))
plt.title('Frequency of Each Issue Type')
plt.xlabel('Issue Type')
plt.ylabel('Count')
plt.show()

In [ ]:
application_counts.plot(kind='bar', figsize=(14, 7))
plt.title('Frequency of Each Application Type')
plt.xlabel('Issue Type')
plt.ylabel('Count')
plt.show()